# lstm GRU
원본: https://www.kaggle.com/hrshuvo/dnn-lstm-tpu
GPU참고: https://www.kaggle.com/brennolins/ventilator-pressure-prediction-lstm

In [1]:
### 참고 EDA 
## 01 To learn or not to learn? : https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964
print("01 To learn or not to learn?: ", "https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964")
print()

## 01-1) PyTorch LSTM with TensorFlow-like initialization
print("01-1) PyTorch LSTM with TensorFlow-like initialization", "https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization")
print()

## 01-2) [01-1의 오리진?] Ventilator Pressure | EDA | LSTM 0.189 🔥
print("Ventilator Pressure | EDA | LSTM 0.189 🔥", "https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook")
print()


## 02-1) finetune of Tensorflow Bidirectional LSTM
print("02-1) finetune of Tensorflow Bidirectional LSTM", 'https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm')
print()

## 02-2 Ensemble Folds with MEDIAN - [0.153]
print("02-2) Ensemble Folds with MEDIAN - [0.153]", "https://www.kaggle.com/cdeotte/ensemble-folds-with-median-0-153")
print()


## 03 finetune of Tensorflow Bidirectional LSTM
print("03. finetune of Tensorflow Bidirectional LSTM", 'https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm')
print()

## 용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer] : https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer
print("용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer]: ", "https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer")
print()

##  Google Ventilator LOFO Feature Importance
print("Google Ventilator LOFO Feature Importance", "https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance")
print()

01 To learn or not to learn?:  https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/278964

01-1) PyTorch LSTM with TensorFlow-like initialization https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization

Ventilator Pressure | EDA | LSTM 0.189 🔥 https://www.kaggle.com/dmitryuarov/ventilator-pressure-eda-lstm-0-189/notebook

02-1) finetune of Tensorflow Bidirectional LSTM https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm

02-2) Ensemble Folds with MEDIAN - [0.153] https://www.kaggle.com/cdeotte/ensemble-folds-with-median-0-153

03. finetune of Tensorflow Bidirectional LSTM https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm

용담님 - Single Bi-LSTM Model Pressure Predict(GPU) [Infer]:  https://www.kaggle.com/emphymachine/single-bi-lstm-model-pressure-predict-gpu-infer

Google Ventilator LOFO Feature Importance https://www.kaggle.com/aerdem4/google-ventilator-lofo-feature-importance



## Preperation

### Import 해야지?

In [2]:
## gpu 정보
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [3]:
!pip install einops

In [4]:
# Google Ventilaotr 내에서 마운트 및 로드 

base_path = '../input/ventilator-pressure-prediction/'

import numpy as np
import pandas as pd
import gc
import os


from einops.layers.keras import Rearrange

from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Bidirectional, Add, GRU, BatchNormalization, Conv1D, Embedding
from tensorflow.keras.layers import Activation, Dense, LayerNormalization, LSTM, Multiply, Dense, Dropout, Input, Concatenate
from tensorflow.keras.activations import selu

from scipy.signal import butter, filtfilt
from pickle import dump

from IPython.display import display
from warnings import filterwarnings
filterwarnings('ignore')

num_workers = os.cpu_count()
num_workers

2021-11-03 04:45:32.468593: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-03 04:45:32.468708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.4.1


4

In [5]:
# debug = True
# n = 100 * 1024 if debug else None

# ss = pd.read_csv(base_path + 'sample_submission.csv', nrows= n)
train_org = pd.read_csv(base_path + 'train.csv')
test_org = pd.read_csv(base_path + 'test.csv')

# ss = pd.read_csv(base_path + 'sample_submission.csv')
# train_org = pd.read_csv(base_path + 'train.csv')
# test_org = pd.read_csv(base_path + 'test.csv')

## Save Memory

In [6]:
# 차오르는 RAM을 아껴볼까?
# save_ram : 말 그대로 RAM을 아끼기 위해, 데이터 다이어트를 할 수 있는 코드이다. 
# 출처1:https://www.kaggle.com/dmitryuarov/tps-soft-voting-xgb-cb-lgbm#Basic-information
# 출처2: https://www.kaggle.com/rinnqd/reduce-memory-usage
# 지금 당장 쓰지 않을 것이다. 
# ㄴ사유: 데이터 타입으로 표현되는 숫자가 다르기 때문에, 스케일링 직후(표현되는 스케일이 비슷하기 때문에), 모델에 넣기 전에 하는 것이 적절하다. 

def save_ram(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"최적화 이후 메모리 사용량: {round(end_mem, 2)}MB")
    print(f"메모리 감소비율: {round(100*(start_mem - end_mem)/(start_mem), 2)}%")

    return df

In [7]:
train_org = save_ram(train_org, verbose=True)

최적화 이후 메모리 사용량: 97.86MB
메모리 감소비율: 73.44%


In [8]:
train_org.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083313,0,5.835938
1,2,1,20,50,0.033661,18.390625,0,5.906250
2,3,1,20,50,0.067505,22.515625,0,7.875000
3,4,1,20,50,0.101562,22.812500,0,11.742188
4,5,1,20,50,0.135742,25.359375,0,12.234375


In [9]:
test_org = save_ram(test_org, verbose = True)

최적화 이후 메모리 사용량: 57.56MB
메모리 감소비율: 73.21%


In [10]:
test_org.head()

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031891,7.515625,0
2,3,0,5,20,0.063843,14.648438,0
3,4,0,5,20,0.095764,21.234375,0
4,5,0,5,20,0.127686,26.328125,0


## add_features

In [11]:
def add_features(df):
    df['R1'] = df['R'].copy()
    df['C1'] = df['C'].copy()

    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    df = df.fillna(0)
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']

    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)

    g = df.groupby('breath_id')['u_in'].apply(pd.Series)
    
    df['ewm_u_in_mean'] = g.ewm(halflife=9).mean()\
                           .reset_index(level=0, drop=True)
                           

    # print("====add_more_features_Begins===")
    # df['ewm_u_in_std'] = g.ewm(halflife=10).std()\
    #                       .reset_index(level=0, drop=True)
    # df['ewm_u_in_corr'] = g.ewm(halflife=10).corr()\
    #                        .reset_index(level=0, drop=True)
    # df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
    #                          .reset_index(level=0, drop=True)
    # df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
    #                         .reset_index(level=0, drop=True)
    # df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
    #                         .reset_index(level=0, drop=True)

    # df['expand_mean'] = g.expanding(2).mean()\
    #                      .reset_index(level=0, drop=True)
    # df['expand_max'] = g.expanding(2).max()\
    #                     .reset_index(level=0, drop=True)
    # df['expand_std'] = g.expanding(2).std()\
    #                     .reset_index(level=0, drop=True)
    # df['R_new'] = df["R"].apply(lambda val: 1 / val**4)
    # df = df.fillna(0)
    # print("====add_more_features_Completed===")
    
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"})\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
    
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")
    df = df.fillna(0)
    df['R_new']=df['R'].apply(lambda val: 1 / (val**4))
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)

    df['R2'] = df['R1'].copy()
    df['C2'] = df['C1'].copy()
    print("Step-8...Completed")
    
    return df

In [12]:
train = add_features(train_org)
del train_org
print(train.shape)

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed
(6036000, 78)


In [13]:
test = add_features(test_org)
print(test.shape)
del test_org

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed
(4024000, 77)


In [14]:
train = save_ram(train, verbose=True)

최적화 이후 메모리 사용량: 805.89MB
메모리 감소비율: 55.41%


In [15]:
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,R1,C1,cross,cross2,...,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,R2,C2
0,1,1,0.000000,0.083313,0,5.835938,20,50,0.0,0.0,...,0,1,0,0,0,0,0,0,20,50
1,2,1,0.033661,18.390625,0,5.906250,20,50,0.0,0.0,...,0,1,0,0,0,0,0,0,20,50
2,3,1,0.067505,22.515625,0,7.875000,20,50,0.0,0.0,...,0,1,0,0,0,0,0,0,20,50
3,4,1,0.101562,22.812500,0,11.742188,20,50,0.0,0.0,...,0,1,0,0,0,0,0,0,20,50
4,5,1,0.135742,25.359375,0,12.234375,20,50,0.0,0.0,...,0,1,0,0,0,0,0,0,20,50


In [16]:
test = save_ram(test)

최적화 이후 메모리 사용량: 529.59MB
메모리 감소비율: 55.77%


In [17]:
test.head()

,id,breath_id,time_step,u_in,u_out,R1,C1,cross,cross2,area,...,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,R2,C2
0,1,0,0.000000,0.000000,0,5,20,0.0,0.0,0.000000,...,0,0,0,0,0,0,1,0,5,20
1,2,0,0.031891,7.515625,0,5,20,0.0,0.0,0.239624,...,0,0,0,0,0,0,1,0,5,20
2,3,0,0.063843,14.648438,0,5,20,0.0,0.0,1.174805,...,0,0,0,0,0,0,1,0,5,20
3,4,0,0.095764,21.234375,0,5,20,0.0,0.0,3.207031,...,0,0,0,0,0,0,1,0,5,20
4,5,0,0.127686,26.328125,0,5,20,0.0,0.0,6.570312,...,0,0,0,0,0,0,1,0,5,20


### 필요없는 feature drop


In [18]:
## target
targets = train[['pressure']].to_numpy().reshape(-1, 80)
targets.shape

(75450, 80)

In [19]:
## train
train.drop(['pressure', 'id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

In [20]:
## 확인
print(train.shape)
train.head(1)

(6036000, 69)


,time_step,u_in,u_out,R1,C1,cross,cross2,area,time_step_cumsum,u_in_cumsum,...,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,R2,C2
0,0.0,0.083313,0,20,50,0.0,0.0,0.0,0.0,0.083313,...,0,1,0,0,0,0,0,0,20,50


In [21]:
## Test
test = test.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame',
                  'breath_id_lag2same'], axis=1)

In [22]:
## 확인
print(test.shape)
test.head(1)

(4024000, 69)


,time_step,u_in,u_out,R1,C1,cross,cross2,area,time_step_cumsum,u_in_cumsum,...,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,R2,C2
0,0.0,0.0,0,5,20,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,5,20


## scaler

In [23]:
train.columns # 수정

Index(['time_step', 'u_in', 'u_out', 'R1', 'C1', 'cross', 'cross2', 'area',
       'time_step_cumsum', 'u_in_cumsum', 'u_in_lag1', 'u_out_lag1',
       'u_in_lag_back1', 'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2',
       'u_in_lag_back2', 'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3',
       'u_in_lag_back3', 'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4',
       'u_in_lag_back4', 'u_out_lag_back4', 'breath_id__u_in__max',
       'breath_id__u_in__mean', 'breath_id__u_in__diffmax',
       'breath_id__u_in__diffmean', 'u_in_diff1', 'u_out_diff1', 'u_in_diff2',
       'u_out_diff2', 'u_in_diff3', 'u_out_diff3', 'u_in_diff4', 'u_out_diff4',
       'u_in_cummean', 'breath_id__u_in_lag', 'breath_id__u_in_lag2',
       'time_step_diff', 'ewm_u_in_mean', '15_in_sum', '15_in_min',
       '15_in_max', '15_in_mean', 'u_in_lagback_diff1', 'u_out_lagback_diff1',
       'u_in_lagback_diff2', 'u_out_lagback_diff2', 'R_new', 'R_20', 'R_5',
       'R_50', 'C_10', 'C_20', 'C_50', 'R__C_20__10', 'R__C_20

In [24]:
## 수정
train.columns
train_rc = train[['R2', 'C2']]
test_rc = test[['R2', 'C2']]
columnss = ['time_step', 'u_in', 'u_out', 'R1', 'C1', 'cross', 'cross2', 'area',
       'time_step_cumsum', 'u_in_cumsum', 'u_in_lag1', 'u_out_lag1',
       'u_in_lag_back1', 'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2',
       'u_in_lag_back2', 'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3',
       'u_in_lag_back3', 'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4',
       'u_in_lag_back4', 'u_out_lag_back4', 'breath_id__u_in__max',
       'breath_id__u_in__mean', 'breath_id__u_in__diffmax',
       'breath_id__u_in__diffmean', 'u_in_diff1', 'u_out_diff1', 'u_in_diff2',
       'u_out_diff2', 'u_in_diff3', 'u_out_diff3', 'u_in_diff4', 'u_out_diff4',
       'u_in_cummean', 'breath_id__u_in_lag', 'breath_id__u_in_lag2',
       'time_step_diff', 'ewm_u_in_mean', '15_in_sum', '15_in_min',
       '15_in_max', '15_in_mean', 'u_in_lagback_diff1', 'u_out_lagback_diff1',
       'u_in_lagback_diff2', 'u_out_lagback_diff2', 'R_new', 'R_20', 'R_5',
       'R_50', 'C_10', 'C_20', 'C_50', 'R__C_20__10', 'R__C_20__20',
       'R__C_20__50', 'R__C_50__10', 'R__C_50__20', 'R__C_50__50',
       'R__C_5__10', 'R__C_5__20', 'R__C_5__50', 'R2', 'C2']

In [25]:
sc = RobustScaler()
train = sc.fit_transform(train[columnss])
test = sc.transform(test[columnss]) ## np.ndarray

In [26]:
train

array([[-0.9893, -0.9375, -1.    , ...,  0.    ,  0.    ,  0.75  ],
       [-0.9644,  3.049 , -1.    , ...,  0.    ,  0.    ,  0.75  ],
       [-0.9385,  3.947 , -1.    , ...,  0.    ,  0.    ,  0.75  ],
       ...,
       [ 0.9556, -0.616 ,  0.    , ...,  0.    ,  0.6665, -0.25  ],
       [ 0.9805, -0.678 ,  0.    , ...,  0.    ,  0.6665, -0.25  ],
       [ 1.006 , -0.633 ,  0.    , ...,  0.    ,  0.6665, -0.25  ]],
      dtype=float16)

In [27]:
train_rc.to_numpy()

array([[20, 50],
       [20, 50],
       [20, 50],
       ...,
       [50, 10],
       [50, 10],
       [50, 10]], dtype=int8)

In [28]:
train = np.concatenate((train, train_rc), axis = 1)

In [29]:
test = np.concatenate((test, test_rc), axis = 1)

In [30]:
train = train.reshape(-1, 80, train.shape[-1]).astype('float32')
train.shape

(75450, 80, 71)

In [31]:
test = test.reshape(-1, 80, train.shape[-1]).astype('float32')
test.shape

(50300, 80, 71)

In [32]:
pressure = targets.squeeze().reshape(-1, 1).astype('float32')

P_MIN = np.min(pressure)
P_MAX = np.max(pressure)
P_STEP = (pressure[1] -  pressure[0])[0]

print(f"MIN Pressure: {P_MIN}")
print(f"MAX Pressure: {P_MAX}")
print(f"Pressure step: {P_STEP}")
print(f"Unique Values: {np.unique(pressure).shape[0]}")

MIN Pressure: -1.8955078125
MAX Pressure: 64.8125
Pressure step: 0.0703125
Unique Values: 950


In [33]:
del pressure

In [34]:
gc.collect()

161

## TPU or NOT?

In [35]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

2021-11-03 04:49:22.763711: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-03 04:49:22.767564: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-03 04:49:22.767606: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-03 04:49:22.767636: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (138e9274ac52): /proc/driver/nvidia/version does not exist
2021-11-03 04:49:22.771445: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Running on TPU: grpc://10.0.0.2:8470
Batch Size: 512


## Model

In [36]:

def rc_to_index(r,c):
    ## 가중치?
    #r_map = { 5: 0, 20: 1, 50: 2}
    # # #c_map = {10: 0, 20: 1, 50: 2}
    # r = (r== 5).float()*1 + (r==20).float()*2 + (r==50).float()*3
    # c = (c==10).float()*1 + (c==20).float()*2 + (c==50).float()*3
    # r = r.long()
    
    # r = (r== 5)*1 + (r==20)*2 + (r==50)*3
    # c = (c==10)*1 + (c==20)*2 + (c==50)*3

    r1 = 0
    r2 = 0
    r3 = 0
    if r == 5:
        r1 += 1
    if r == 20:
        r2 += 2
    if r == 50:
        r3 += 3

    r = r1 + r2 + r3

    
    c1 = 0
    c2 = 0
    c3 = 0
    if c == 5:
        c1 += 1
    if c == 20:
        c2 += 2
    if c == 50:
        c3 += 3

    c = c1 + c2 + c3

    # c = c.long()
    # if r == 5:
    #     r = 0
    #     r = r.astype('float')
    # elif r == 10:
    #     r = 1
    #     r = r.astype('float')
    # elif r == 50:
    #     r = 1
    #     r = r.astype('float')

    
    # if c == 10:
    #     c = 0
    #     c= c.astype('float')
    # elif c == 20:
    #     c = 1
    #     c = c.astype('float')
    # elif c == 50:
    #     c = 1
    #     c = c.astype('float')

    return r,c

In [44]:
def d_model():

    x_input = Input(shape=(train.shape[-2:]))

    r_embed = Embedding(4, 2)
    c_embed = Embedding(4, 2)

    r = r_embed(x_input[:, :, -2])
    c = c_embed(x_input[:, :, -1])

    seq= Concatenate(axis=2)([r, c, x_input[:, :, :-2]])

    x_conv1d = Conv1D(256, kernel_size=5, strides=1, padding='same')(seq)
    x_LN = LayerNormalization()(x_conv1d)
    x_selu = selu(x_LN)
    
    x1 = Bidirectional(LSTM(units = 768, return_sequences = True))(x_selu)
    x2 = Bidirectional(LSTM(units = 512, return_sequences = True))(x1)

    ## z2랑
    x3 = Bidirectional(LSTM(units = 384, return_sequences = True))(x2)

    ## z3랑
    x4 = Bidirectional(LSTM(units = 256, return_sequences = True))(x3)

    ## z4랑 
    x5 = Bidirectional(LSTM(units=128, return_sequences=True))(x4)
    
    z2 = Bidirectional(GRU(units=384, return_sequences=True))(x2)
    
    z31 = Multiply()([x3, z2])
    z31 = BatchNormalization()(z31)

    z3 = Bidirectional(GRU(units=256, return_sequences=True))(z31)
    
    z41 = Multiply()([x4, z3])
    z41 = BatchNormalization()(z41)
    
    z4 = Bidirectional(GRU(units=128, return_sequences=True))(z41)
    
    z51 = Multiply()([x5, z4])
    z51 = BatchNormalization()(z51)
    
    z5 = Bidirectional(GRU(units=64, return_sequences=True))(z51)
    
    x = Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = Dense(units=128, activation='selu')(x)
    
    x_output = Dense(units=1)(x)


    model = Model(inputs=x_input, outputs=x_output, name='BLG_Model')

    return model

In [45]:
model = d_model()
model.summary()

Model: "BLG_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 80, 71)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_6 (Sli (None, 80)           0           input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_7 (Sli (None, 80)           0           input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 80, 2)        8           tf.__operators__.getitem_6[0][0] 
__________________________________________________________________________________________

In [46]:
## 참고: https://www.kaggle.com/brennolins/ventilator-pressure-prediction-lstm

## Train & Predict

In [48]:
with tpu_strategy.scope():
    
    VERBOSE = 0
    test_preds = []
    kf = KFold(n_splits=5, shuffle=True, random_state=624)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        
        model = d_model()
        model.compile(optimizer="adam", loss="mae")

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.85, 
                               patience=7, verbose=VERBOSE)
        
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        chk_point = ModelCheckpoint(f'./Bidirect_LSTM_model_{fold+1}C.h5', options=save_locally, 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=30, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        model.fit(X_train, y_train, 
                  validation_data=(X_valid, y_valid), 
                  epochs=400,
                  verbose=VERBOSE,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = load_model(f'./Bidirect_LSTM_model_{fold+1}C.h5', options=load_locally)
        
        y_true = y_valid.squeeze().reshape(-1, 1)
        y_pred = model.predict(X_valid, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1)
        score = mean_absolute_error(y_true, y_pred)
        print(f"Fold-{fold+1} - Score: {score}")
        
        test_preds.append(model.predict(test, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1).squeeze())

        # if fold >= 1:
            # break

2021-11-03 05:15:00.404553: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1371379200 exceeds 10% of free system memory.


Fold-1 - Score: 0.1668313592672348


2021-11-03 06:11:29.057463: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1142816000 exceeds 10% of free system memory.
2021-11-03 06:11:46.532519: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1371379200 exceeds 10% of free system memory.


Fold-2 - Score: 0.16512945294380188
Fold-3 - Score: 0.16590960323810577
Fold-4 - Score: 0.16904006898403168
Fold-5 - Score: 0.1686827540397644


In [ ]:
# 1350 learning start   
# 1519 fold1 score 0.1668313592672348
# 1612 fold2 score 0.16512945294380188
# 1654 fold3 score 0.16590960323810577
# 1815 fold4 score 0.16904006898403168
# 1900 fold5 score 0.1686827540397644

In [50]:
ss = pd.read_csv(base_path + 'sample_submission.csv')

In [51]:
ss["pressure"] = np.median(np.vstack(test_preds),axis=0)
ss["pressure"] = np.round((ss.pressure - P_MIN)/P_STEP) * P_STEP + P_MIN
ss["pressure"] = np.clip(ss.pressure, P_MIN, P_MAX)
ss.to_csv('blg_tpu_median_conv1d256_submission.csv', index=False)
ss.to_csv('blg_tpu_conv1d256_submission.csv', index=False)

In [52]:
df_folds = pd.DataFrame(test_preds)
df_folds = df_folds.T
## 오래걸림
df_folds.columns = ['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5']
df_folds.to_csv('kb_conv1d256_5folds.csv', index=False)
df_folds.shape

(4024000, 5)

In [ ]:
ss

## Submit

In [ ]:
base_path

In [ ]:
sample = pd.read_csv(base_path + 'sample_submission.csv')

In [ ]:
sample["pressure"] = np.median(np.vstack(test_preds),axis=0)
sample["pressure"] = np.round((sample.pressure - P_MIN)/P_STEP) * P_STEP + P_MIN
sample["pressure"] = np.clip(sample.pressure, P_MIN, P_MAX)
sample.to_csv('blg_tpu_median_conv1d_submission.csv', index=False)
sample.to_csv('blg_tpu_conv1d_submission.csv', index=False)